In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup,WhisperForConditionalGeneration
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model


/hdd/Gothi_raj/envs/wt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model_name_or_path = 'openai/whisper-medium'

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path,cache_dir = '/hdd/Gothi_raj/HF_model')
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [3]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 773,295,104 || trainable%: 1.2204


In [4]:
model

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1024)
          (layers): ModuleList(
            (0-23): 24 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=32, bias=False)
                  )
                  (lora_B): M

In [9]:
for name,parameters in model.named_parameters():
    if 'layer_norm' in name:
        parameters.requires_grad = True

    print(name,parameters.requires_grad)

base_model.model.model.encoder.conv1.weight False
base_model.model.model.encoder.conv1.bias False
base_model.model.model.encoder.conv2.weight False
base_model.model.model.encoder.conv2.bias False
base_model.model.model.encoder.embed_positions.weight False
base_model.model.model.encoder.layers.0.self_attn.k_proj.weight False
base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.weight False
base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.bias False
base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight True
base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight True
base_model.model.model.encoder.layers.0.self_attn.q_proj.base_layer.weight False
base_model.model.model.encoder.layers.0.self_attn.q_proj.base_layer.bias False
base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight True
base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_B.default.weight True
base_model.model.m